## Boosting

Boosting is a machine learning meta-algorithm for reducing bias in supervised learning. The procedure is intended to "boost" the performance of a weak learner (a classifier that is only slightly correlated with the true classification) into a strong learner (a classifier that is arbitrarily well-correlated with the true classification).

While boosting is not algorithmically constrained, most boosting algorithms consist of iteratively learning weak classifiers with respect to a distribution and adding them to a final strong classifier. When they are added, they are typically weighted in some way that is usually related to the weak learners' accuracy. After a weak learner is added, the data is reweighted:  examples that are misclassified gain weight (and examples that are classified correctly may lose weight). Thus, future weak learners focus more on the examples that previous weak learners misclassified.

The main variation between many boosting algorithms is their method of weighting training data points and hypotheses. AdaBoost is very popular and perhaps the most significant historically as it was the first algorithm that could adapt to the weak learners.

In [1]:
# (c) 2014 Reid Johnson
#
# Modified from:
# sklearn (https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/ensemble/weight_boosting.py)
#
# Generates a "boosted" ensemble of base models.

import copy
import operator
import sys, math, random
import numpy

from sklearn.base import ClassifierMixin
from sklearn.tree import DecisionTreeClassifier

class AdaBoostClassifier(ClassifierMixin):
    """An AdaBoost classifier.

    An AdaBoost [1] classifier is a meta-estimator that begins by fitting a
    classifier on the original dataset and then fits additional copies of the
    classifier on the same dataset but where the weights of incorrectly
    classified instances are adjusted such that subsequent classifiers focus
    more on difficult cases.

    This class implements the algorithm known as AdaBoost-SAMME [2].

    Parameters
    ----------
    base_estimator : object, optional (default=DecisionTreeClassifier)
        The base estimator from which the boosted ensemble is built.
        Support for sample weighting is required, as well as proper `classes_`
        and `n_classes_` attributes.

    n_estimators : integer, optional (default=50)
        The maximum number of estimators at which boosting is terminated.
        In case of perfect fit, the learning procedure is stopped early.

    learning_rate : float, optional (default=1.)
        Learning rate shrinks the contribution of each classifier by
        ``learning_rate``. There is a trade-off between ``learning_rate`` and
        ``n_estimators``.

    algorithm : {'SAMME', 'SAMME.R'}, optional (default='SAMME.R')
        If 'SAMME.R' then use the SAMME.R real boosting algorithm.
        ``base_estimator`` must support calculation of class probabilities.
        If 'SAMME' then use the SAMME discrete boosting algorithm.
        The SAMME.R algorithm typically converges faster than SAMME,
        achieving a lower test error with fewer boosting iterations.

    random_state : int, RandomState instance or None, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`.

    Attributes
    ----------
    `estimators_` : list of classifiers
        The collection of fitted sub-estimators.

    `classes_` : array of shape = [n_classes]
        The classes labels.

    `n_classes_` : int
        The number of classes.

    `estimator_weights_` : array of floats
        Weights for each estimator in the boosted ensemble.

    `estimator_errors_` : array of floats
        Classification error for each estimator in the boosted
        ensemble.

    `feature_importances_` : array of shape = [n_features]
        The feature importances if supported by the ``base_estimator``.

    References
    ----------
    .. [1] Y. Freund, R. Schapire, "A Decision-Theoretic Generalization of
           on-Line Learning and an Application to Boosting", 1995.

    .. [2] J. Zhu, H. Zou, S. Rosset, T. Hastie, "Multi-class AdaBoost", 2009.

    """
    def __init__(self,
                 base_estimator=DecisionTreeClassifier(max_depth=1),
                 n_estimators=50,
                 estimator_params=tuple(),
                 learning_rate=1.,
                 algorithm='SAMME.R'):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.estimator_params = estimator_params
        self.learning_rate = learning_rate
        self.algorithm = algorithm

    def _make_estimator(self, append=True):
        """Make and configure a copy of the `base_estimator_` attribute.

        Warning: This method should be used to properly instantiate new
        sub-estimators.
        """
        estimator = copy.deepcopy(self.base_estimator)
        estimator.set_params(**dict((p, getattr(self, p))
                                    for p in self.estimator_params))

        if append:
            self.estimators_.append(estimator)

        return estimator

    def fit(self, X, y, sample_weight=None):
        # Check parameters.
        if self.learning_rate <= 0:
            raise ValueError("learning_rate must be greater than zero.")

        if sample_weight is None:
            # Initialize weights to 1 / n_samples.
            sample_weight = np.empty(X.shape[0], dtype=np.float)
            sample_weight[:] = 1. / X.shape[0]
        else:
            # Normalize existing weights.
            sample_weight = sample_weight / sample_weight.sum(dtype=np.float64)

        # Check that the sample weights sum is positive.
        if sample_weight.sum() <= 0:
            raise ValueError(
                "Attempting to fit with a non-positive "
                "weighted number of samples.")

        # Clear any previous fit results.
        self.estimators_ = []
        self.estimator_weights_ = np.zeros(self.n_estimators, dtype=np.float)
        self.estimator_errors_ = np.ones(self.n_estimators, dtype=np.float)

        for iboost in range(self.n_estimators):
            #print 'Iteration [%s]' % (iboost)

            # Fit the estimator.
            estimator = self._make_estimator()
            estimator.fit(X, y, sample_weight=sample_weight)

            if iboost == 0:
                self.classes_ = getattr(estimator, 'classes_', None)
                self.n_classes_ = len(self.classes_)

            # Generate estimator predictions.
            y_pred = estimator.predict(X)

            # Instances incorrectly classified.
            incorrect = y_pred != y

            # Error fraction.
            estimator_error = np.mean(
                np.average(incorrect, weights=sample_weight, axis=0))

            # Boost weight using multi-class AdaBoost SAMME alg.
            estimator_weight = self.learning_rate * (
                np.log((1. - estimator_error) / estimator_error) +
                np.log(self.n_classes_ - 1.))

            # Only boost the weights if there is another iteration of fitting.
            if not iboost == self.n_estimators - 1:
                # Only boost positive weights (exponential loss).
                sample_weight *= np.exp(estimator_weight * incorrect *
                                        ((sample_weight > 0) |
                                         (estimator_weight < 0)))

            self.estimator_weights_[iboost] = estimator_weight
            self.estimator_errors_[iboost] = estimator_error

    def _check_fitted(self):
        if not hasattr(self, "estimators_"):
            raise ValueError("Call 'fit' first.")

    def predict(self, X):
        X = numpy.array(X)
        N, d = X.shape
        pred = numpy.zeros(N)
        for estimator, w in zip(self.estimators_, self.estimator_weights_):
            pred += estimator.predict(X) * w
        pred /= self.estimator_weights_.sum()

        return pred

    def decision_function(self, X):
        """Compute the decision function of ``X``.

        Parameters
        ----------
        X : array-like of shape = [n_samples, n_features]
            The input samples.

        Returns
        -------
        score : array, shape = [n_samples, k]
            The decision function of the input samples. The order of
            outputs is the same of that of the `classes_` attribute.
            Binary classification is a special cases with ``k == 1``,
            otherwise ``k==n_classes``. For binary classification,
            values closer to -1 or 1 mean more like the first or second
            class in ``classes_``, respectively.
        """
        self._check_fitted()
        X = np.asarray(X)

        n_classes = self.n_classes_
        classes = self.classes_[:, np.newaxis]
        pred = None

        if self.algorithm == 'SAMME.R':
            # The weights are all 1. for SAMME.R
            pred = sum(_samme_proba(estimator, n_classes, X)
                       for estimator in self.estimators_)
        else:   # self.algorithm == "SAMME"
            pred = sum((estimator.predict(X) == classes).T * w
                       for estimator, w in zip(self.estimators_,
                                               self.estimator_weights_))

        pred /= self.estimator_weights_.sum()
        if n_classes == 2:
            pred[:, 0] *= -1
            return pred.sum(axis=1)
        return pred

    def predict(self, X):
        """Predict classes for X.

        The predicted class of an input sample is computed as the weighted mean
        prediction of the classifiers in the ensemble.

        Parameters
        ----------
        X : array-like of shape = [n_samples, n_features]
            The input samples.

        Returns
        -------
        y : array of shape = [n_samples]
            The predicted classes.
        """
        pred = self.decision_function(X)

        if self.n_classes_ == 2:
            return self.classes_.take(pred > 0, axis=0)

        return self.classes_.take(np.argmax(pred, axis=1), axis=0)

    def predict_proba(self, X):
        """Predict class probabilities for X.

        The predicted class probabilities of an input sample is computed as
        the weighted mean predicted class probabilities of the classifiers
        in the ensemble.

        Parameters
        ----------
        X : array-like of shape = [n_samples, n_features]
            The input samples.

        Returns
        -------
        p : array of shape = [n_samples]
            The class probabilities of the input samples. The order of
            outputs is the same of that of the `classes_` attribute.
        """
        X = np.asarray(X)
        n_classes = self.n_classes_

        if self.algorithm == 'SAMME.R':
            # The weights are all 1. for SAMME.R
            proba = sum(_samme_proba(estimator, n_classes, X)
                        for estimator in self.estimators_)
        else:   # self.algorithm == "SAMME"
            proba = sum(estimator.predict_proba(X) * w
                        for estimator, w in zip(self.estimators_,
                                                self.estimator_weights_))

        proba /= self.estimator_weights_.sum()
        proba = np.exp((1. / (n_classes - 1)) * proba)
        normalizer = proba.sum(axis=1)[:, np.newaxis]
        normalizer[normalizer == 0.0] = 1.0
        proba /= normalizer

        return proba

def _samme_proba(estimator, n_classes, X):
    """Calculate algorithm 4, step 2, equation c) of Zhu et al [1].

    References
    ----------
    .. [1] J. Zhu, H. Zou, S. Rosset, T. Hastie, "Multi-class AdaBoost", 2009.

    """
    proba = estimator.predict_proba(X)

    # Displace zero probabilities so the log is defined.
    # Also fix negative elements which may occur with
    # negative sample weights.
    proba[proba <= 0] = 1e-5
    log_proba = np.log(proba)

    return (n_classes - 1) * (log_proba - (1. / n_classes)
                           * log_proba.sum(axis=1)[:, np.newaxis])

In [2]:
import copy
import operator
import sys, math, random
import numpy

from sklearn.base import ClassifierMixin
from sklearn.tree import DecisionTreeClassifier

class LogitBoostClassifier(ClassifierMixin):
    """A LogitBoost classifier.

    A LogitBoost [1] classifier is a meta-estimator that begins by fitting a
    classifier on the original dataset and then fits additional copies of the
    classifier on the same dataset but where the weights of incorrectly
    classified instances are adjusted such that subsequent classifiers focus
    more on difficult cases.

    Parameters
    ----------
    base_estimator : object, optional (default=DecisionTreeClassifier)
        The base estimator from which the boosted ensemble is built.
        Support for sample weighting is required, as well as proper `classes_`
        and `n_classes_` attributes.

    n_estimators : integer, optional (default=50)
        The maximum number of estimators at which boosting is terminated.
        In case of perfect fit, the learning procedure is stopped early.

    learning_rate : float, optional (default=1.)
        Learning rate shrinks the contribution of each classifier by
        ``learning_rate``. There is a trade-off between ``learning_rate`` and
        ``n_estimators``.

    algorithm : {'SAMME', 'SAMME.R'}, optional (default='SAMME.R')
        If 'SAMME.R' then use the SAMME.R real boosting algorithm.
        ``base_estimator`` must support calculation of class probabilities.
        If 'SAMME' then use the SAMME discrete boosting algorithm.
        The SAMME.R algorithm typically converges faster than SAMME,
        achieving a lower test error with fewer boosting iterations.

    random_state : int, RandomState instance or None, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`.

    Attributes
    ----------
    `estimators_` : list of classifiers
        The collection of fitted sub-estimators.

    `classes_` : array of shape = [n_classes]
        The classes labels.

    `n_classes_` : int
        The number of classes.

    `estimator_weights_` : array of floats
        Weights for each estimator in the boosted ensemble.

    `estimator_errors_` : array of floats
        Classification error for each estimator in the boosted
        ensemble.

    `feature_importances_` : array of shape = [n_features]
        The feature importances if supported by the ``base_estimator``.

    References
    ----------
    .. [1] J. Friedman, T. Hastie, R. Tibshirani, "Additive Logistic Regression: 
           A Statistical View of Boosting", 2000.

    """
    def __init__(self,
                 base_estimator=DecisionTreeClassifier(max_depth=1),
                 n_estimators=50,
                 estimator_params=tuple(),
                 learning_rate=1.,
                 algorithm='SAMME.R'):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.estimator_params = estimator_params
        self.learning_rate = learning_rate
        self.algorithm = algorithm

    def _make_estimator(self, append=True):
        """Make and configure a copy of the `base_estimator_` attribute.

        Warning: This method should be used to properly instantiate new
        sub-estimators.
        """
        estimator = copy.deepcopy(self.base_estimator)
        estimator.set_params(**dict((p, getattr(self, p))
                                    for p in self.estimator_params))

        if append:
            self.estimators_.append(estimator)

        return estimator

    def fit(self, X, y, sample_weight=None):
        # Check parameters.
        if self.learning_rate <= 0:
            raise ValueError("learning_rate must be greater than zero.")

        if sample_weight is None:
            # Initialize weights to 1 / n_samples.
            sample_weight = np.empty(X.shape[0], dtype=np.float)
            sample_weight[:] = 1. / X.shape[0]
        else:
            # Normalize existing weights.
            sample_weight = sample_weight / sample_weight.sum(dtype=np.float64)

        # Check that the sample weights sum is positive.
        if sample_weight.sum() <= 0:
            raise ValueError(
                "Attempting to fit with a non-positive "
                "weighted number of samples.")

        # Clear any previous fit results.
        self.estimators_ = []
        self.estimator_weights_ = np.zeros(self.n_estimators, dtype=np.float)
        self.estimator_errors_ = np.ones(self.n_estimators, dtype=np.float)

        for iboost in range(self.n_estimators):
            #print 'Iteration [%s]' % (iboost)

            # Fit the estimator.
            estimator = self._make_estimator()
            estimator.fit(X, y, sample_weight=sample_weight)

            if iboost == 0:
                self.classes_ = getattr(estimator, 'classes_', None)
                self.n_classes_ = len(self.classes_)

            # Generate estimator predictions.
            y_pred = estimator.predict(X)

            # Instances incorrectly classified.
            incorrect = y_pred != y

            # Error fraction.
            estimator_error = np.mean(
                np.average(incorrect, weights=sample_weight, axis=0))

            # Boost weight using multi-class AdaBoost SAMME alg.
            estimator_weight = self.learning_rate * (
                np.log((1. - estimator_error) / estimator_error) +
                np.log(self.n_classes_ - 1.))

            # Only boost the weights if there is another iteration of fitting.
            if not iboost == self.n_estimators - 1:
                # Only boost positive weights (logistic loss).
                sample_weight *= np.log(1 + np.exp(estimator_weight * incorrect *
                                        ((sample_weight > 0) |
                                         (estimator_weight < 0))))

            self.estimator_weights_[iboost] = estimator_weight
            self.estimator_errors_[iboost] = estimator_error

    def _check_fitted(self):
        if not hasattr(self, "estimators_"):
            raise ValueError("Call 'fit' first.")

    def predict(self, X):
        X = numpy.array(X)
        N, d = X.shape
        pred = numpy.zeros(N)
        for estimator, w in zip(self.estimators_, self.estimator_weights_):
            pred += estimator.predict(X) * w
        pred /= self.estimator_weights_.sum()

        return pred

    def decision_function(self, X):
        """Compute the decision function of ``X``.

        Parameters
        ----------
        X : array-like of shape = [n_samples, n_features]
            The input samples.

        Returns
        -------
        score : array, shape = [n_samples, k]
            The decision function of the input samples. The order of
            outputs is the same of that of the `classes_` attribute.
            Binary classification is a special cases with ``k == 1``,
            otherwise ``k==n_classes``. For binary classification,
            values closer to -1 or 1 mean more like the first or second
            class in ``classes_``, respectively.
        """
        self._check_fitted()
        X = np.asarray(X)

        n_classes = self.n_classes_
        classes = self.classes_[:, np.newaxis]
        pred = None

        if self.algorithm == 'SAMME.R':
            # The weights are all 1. for SAMME.R
            pred = sum(_samme_proba(estimator, n_classes, X)
                       for estimator in self.estimators_)
        else:   # self.algorithm == "SAMME"
            pred = sum((estimator.predict(X) == classes).T * w
                       for estimator, w in zip(self.estimators_,
                                               self.estimator_weights_))

        pred /= self.estimator_weights_.sum()
        if n_classes == 2:
            pred[:, 0] *= -1
            return pred.sum(axis=1)
        return pred

    def predict(self, X):
        """Predict classes for X.

        The predicted class of an input sample is computed as the weighted mean
        prediction of the classifiers in the ensemble.

        Parameters
        ----------
        X : array-like of shape = [n_samples, n_features]
            The input samples.

        Returns
        -------
        y : array of shape = [n_samples]
            The predicted classes.
        """
        pred = self.decision_function(X)

        if self.n_classes_ == 2:
            return self.classes_.take(pred > 0, axis=0)

        return self.classes_.take(np.argmax(pred, axis=1), axis=0)

    def predict_proba(self, X):
        """Predict class probabilities for X.

        The predicted class probabilities of an input sample is computed as
        the weighted mean predicted class probabilities of the classifiers
        in the ensemble.

        Parameters
        ----------
        X : array-like of shape = [n_samples, n_features]
            The input samples.

        Returns
        -------
        p : array of shape = [n_samples]
            The class probabilities of the input samples. The order of
            outputs is the same of that of the `classes_` attribute.
        """
        X = np.asarray(X)
        n_classes = self.n_classes_

        if self.algorithm == 'SAMME.R':
            # The weights are all 1. for SAMME.R
            proba = sum(_samme_proba(estimator, n_classes, X)
                        for estimator in self.estimators_)
        else:   # self.algorithm == "SAMME"
            proba = sum(estimator.predict_proba(X) * w
                        for estimator, w in zip(self.estimators_,
                                                self.estimator_weights_))

        proba /= self.estimator_weights_.sum()
        proba = np.log(1 + np.exp((1. / (n_classes - 1)) * proba))
        normalizer = proba.sum(axis=1)[:, np.newaxis]
        normalizer[normalizer == 0.0] = 1.0
        proba /= normalizer

        return proba

def _samme_proba(estimator, n_classes, X):
    """Calculate algorithm 4, step 2, equation c) of Zhu et al [1].

    References
    ----------
    .. [1] J. Zhu, H. Zou, S. Rosset, T. Hastie, "Multi-class AdaBoost", 2009.

    """
    proba = estimator.predict_proba(X)

    # Displace zero probabilities so the log is defined.
    # Also fix negative elements which may occur with
    # negative sample weights.
    proba[proba <= 0] = 1e-5
    log_proba = np.log(proba)

    return (n_classes - 1) * (log_proba - (1. / n_classes)
                           * log_proba.sum(axis=1)[:, np.newaxis])

Here, we will demonstrate boosting by using the Iris flower dataset. Thus, we first load and perform some preprocessing on the data. The preprocessing involves altering the target or class variables, which in the Iris dataset are by default represented as strings (nominal values), but for compatibility reasons need to be represented as integers (numeric values). We perform this conversion using a label-encoding method available via scikit-learn.

In [3]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

label_encode = True

# Load the Iris flower dataset
fileURL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
iris = pd.read_csv(fileURL, 
                   names=['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width', 'Species'],
                   header=None)
iris = iris.dropna()

X = iris[['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']] # features
labels = iris['Species'] # class

if label_encode:
    # Transform string (nominal) output to numeric
    y = preprocessing.LabelEncoder().fit_transform(labels)
else:
    y = labels

Next, we partition the dataset into non-overlapping training and testing sets, with 40% of the data allocated to the training set and 60% allocated to the testing set. All of the training for the boosting algorithms will be performed on the training set, while the testing set will be used solely to generate predictions, the accuracy of which we will later evaluate. 

In [4]:
from sklearn.model_selection import train_test_split

# The training sets will be used for all training and validation purposes.
# The testing sets will only be used for evaluating the final blended (level 1) classifier.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6)

Our boosting algorithms will iteratatively generate an ensemble of base (weak) learners. At each iteration, the a base learner will generate predictions on the set of instances. The learners accuracy on these instances will be used to weight its predictions in the ensemble and to reweight the instances for the next iteration. AdaBoost and LogitBoost differ in the how they adjust the instance weights at each iteration.

Once the base models are trained, they are combined into an ensemble. This means that their outputs (predictions) are weighted an combined to predict the target or class variable of interest.

In [5]:
abc = AdaBoostClassifier(n_estimators=100)
abc.fit(X_train, y_train)

lbc = LogitBoostClassifier(n_estimators=100)
lbc.fit(X_train, y_train)

Now we can compare the performance of the boosted models to the base (weak) ones:

In [6]:
from sklearn import metrics

### Generate predictions with AdaBoost classifier. ###

score = metrics.accuracy_score(y_test, abc.predict(X_test))
print ('AdaBoost Accuracy = %s' % (score))
print

### Generate predictions with LogitBoost classifier. ###

score = metrics.accuracy_score(y_test, lbc.predict(X_test))
print ('LogitBoost Accuracy = %s' % (score))
print

### Generate predictions with base (non-boosted) classifier. ###

clf = DecisionTreeClassifier().fit(X_train, y_train)
score = metrics.accuracy_score(y_test, clf.predict(X_test))
print ('Base Classifier Accuracy = %s' % (score))
print

import sklearn.ensemble
clf2 = sklearn.ensemble.AdaBoostClassifier(n_estimators=100).fit(X_train, y_train)
score = metrics.accuracy_score(y_test, clf2.predict(X_test))
print ('sklearn AdaBoost Accuracy = %s' % (score))

AdaBoost Accuracy = 0.955555555556
LogitBoost Accuracy = 0.955555555556
Base Classifier Accuracy = 0.977777777778
sklearn AdaBoost Accuracy = 0.944444444444


Notice that by using the method of boosting, we generate a "meta" model that can outperform each of the base models.